In [1]:
print("This is generator")

This is generator


In [2]:
import tensorflow as tf
from tensorflow.keras import layers
import numpy as np
import matplotlib.pyplot as plt
import time
import os


In [3]:
def make_generator_model():
    model = tf.keras.Sequential()
    model.add(layers.Dense(7*7*256, use_bias=False, input_shape=(100,)))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    model.add(layers.Reshape((7, 7, 256)))
    model.add(layers.Conv2DTranspose(128, (5, 5), strides=(1, 1), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsample to 14x14
    model.add(layers.Conv2DTranspose(64, (5, 5), strides=(2, 2), padding='same', use_bias=False))
    model.add(layers.BatchNormalization())
    model.add(layers.LeakyReLU())

    # Upsample to 28x28
    model.add(layers.Conv2DTranspose(1, (5, 5), strides=(2, 2), padding='same', use_bias=False, activation='tanh'))
    return model

In [4]:
def load_mnist_data():
    (x_train, _), (_, _) = tf.keras.datasets.mnist.load_data()
    x_train = x_train.astype('float32') 
    x_train = (x_train - 127.5) / 127.5  # Normalize to [-1, 1]
    x_train = np.expand_dims(x_train, axis=-1)  # Add channel dimension
    return x_train

In [5]:
batch_size = 256

In [6]:
real_images = load_mnist_data()
dataset = tf.data.Dataset.from_tensor_slices(real_images).shuffle(60000).batch(batch_size, drop_remainder=True)

#dataset = tf.data.Dataset.from_tensor_slices(real_images).shuffle(60000).batch(batch_size, drop_reminder = True)


11490434/11490434 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [7]:
@tf.function
def calculate_error_sum(generated_images, real_images):
    errors = tf.reduce_sum(tf.square(generated_images - real_images))
    return errors


In [8]:
noise_dim = 100  # Dimension of noise vector
batch_size = 256  # Batch size
epochs = 100     # Total number of epochs


In [9]:
generator = make_generator_model()



/opt/conda/lib/python3.10/site-packages/keras/src/layers/core/dense.py:87: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [10]:
#generator_optimizer = tf.keras.optimizers.Adam(1e-4)
generator_optimizer = tf.keras.optimizers.Adam(1e-4, beta_1=0.5)


In [11]:
def plot_generated_images(images, epoch):
    plt.figure(figsize=(8, 8))
    for i in range(16):  # Display 16 images
        plt.subplot(4, 4, i + 1)
        plt.imshow(images[i, :, :, 0] * 127.5 + 127.5, cmap='gray')  # Scale back to [0, 255]
        plt.axis('off')
    plt.suptitle(f"Generated Images at Epoch {epoch + 1}")
    plt.show()

In [12]:
@tf.function
def train_generator_step(target_images):
    noise = tf.random.normal([batch_size, noise_dim])

    with tf.GradientTape() as gen_tape:
        generated_images = generator(noise, training=True)
        # Use calculate_error_sum as the generator's loss
        gen_loss = calculate_error_sum(generated_images, target_images)

    gradients_of_generator = gen_tape.gradient(gen_loss, generator.trainable_variables)
    generator_optimizer.apply_gradients(zip(gradients_of_generator, generator.trainable_variables))
    
    return gen_loss

In [13]:
save_dir = "generator_weights"
os.makedirs(save_dir, exist_ok=True)

In [14]:
def train_generator_only(dataset, epochs):
    generator_losses = []

    for epoch in range(epochs):
        start = time.time()

        for target_images in dataset:  # Assuming `dataset` provides target images
            gen_loss = train_generator_step(target_images)
            #print(f"{epoch + 1} gen loss : {gen_loss}")
            generator_losses.append(gen_loss.numpy())

        # Calculate the average loss for this epoch
        avg_gen_loss = sum(generator_losses[-len(dataset):]) / len(dataset)

        # Save the generator model weights only every 10 epochs
        if (epoch + 1) % 10 == 0:
            weight_path = os.path.join(save_dir, f"epoch_{epoch+1}.weights.h5")
            generator.save_weights(weight_path)
            print(f"Saved weights for gen_epoch {epoch+1}: {weight_path}")

        # if (epoch + 1) % 10 == 0:
        #     generator.save_weights(f'generator_weights_epoch_{epoch+1}.weights.h5')

        # Optionally generate and save images
        # display.clear_output(wait=True)
        # generate_and_save_images(generator, epoch + 1, seed)

        print('Time for epoch {} is {} sec'.format(epoch + 1, time.time() - start))
        print("Epoch: {}, Generator Loss: {}".format(epoch + 1, avg_gen_loss))

    # Save the generator losses for each epoch to a file
    loss_file_path = os.path.join(os.getcwd(), "Generator_Only_Loss_No_Discriminator.txt")
    with open(loss_file_path, "w") as f:
        for epoch in range(epochs):
            f.write("Epoch: {}, Generator Loss: {}\n".format(epoch + 1, generator_losses[epoch]))

In [15]:
train_generator_only(dataset, epochs)

Time for epoch 1 is 11.938349962234497 sec
Epoch: 1, Generator Loss: 68422.7661091079
Time for epoch 2 is 5.7776939868927 sec
Epoch: 2, Generator Loss: 54983.40962206197
Time for epoch 3 is 5.808734655380249 sec
Epoch: 3, Generator Loss: 54525.81924412393
Time for epoch 4 is 5.8548903465271 sec
Epoch: 4, Generator Loss: 54356.22482638889
Time for epoch 5 is 5.878767728805542 sec
Epoch: 5, Generator Loss: 54273.267594818375
Time for epoch 6 is 5.91940450668335 sec
Epoch: 6, Generator Loss: 54233.90877069979
Time for epoch 7 is 5.964413166046143 sec
Epoch: 7, Generator Loss: 54191.85440037393
Time for epoch 8 is 6.022669792175293 sec
Epoch: 8, Generator Loss: 54164.81405248398
Time for epoch 9 is 6.057636976242065 sec
Epoch: 9, Generator Loss: 54146.59006076389
Saved weights for gen_epoch 10: generator_weights/epoch_10.weights.h5
Time for epoch 10 is 6.125751733779907 sec
Epoch: 10, Generator Loss: 54125.81974492521
Time for epoch 11 is 6.095349073410034 sec
Epoch: 11, Generator Loss: 54